In [1]:
import pandas as pd
import seaborn as sns
import numpy as np


from sklearn import svm
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit
df = pd.read_csv('../merged_data/ready.csv')
df = df.fillna(0)

In [2]:
df.columns
df = df[['Canadian Citizen', 'No Response(Citizen)', 'Non-Canadian Citizen',
       'Refugee', 'No Longer In Use', 'Occupied', 'Unoccupied',
       'Employed 0-30 Hours', 'Employed 30+ Hours', 'Gr.10 - Gr.12',
       'Gr.7 - Gr.9', 'Homemaker', 'Kindergarten - Gr.6',
       'No Response(Employment)', 'Permanently Unable to Work',
       'Post Secondary Student', 'Preschool', 'Retired', 'Unemployed',
       'Common Law', 'Married', 'Never Married', 'No Response(Marital)',
       'Separated/Divorced', 'Widowed', 'Bicycle',
       'Car/Truck/Van (as Driver)', 'Car/Truck/Van (as Passenger)',
       'No Response(Transportation)', 'Other', 'Public Transit', 'Walk',
       'Catholic', 'No Response(School)', 'Public', 'Assault', 'Break and Enter', 'Homicide',
       'Robbery', 'Sexual Assaults', 'Theft From Vehicle', 'Theft Of Vehicle',
       'Theft Over $5000']]

In [3]:
def train_test_spliter(df, random_seed=0, test_size=0.20, n_splits=1):
    # trainm = preprocessing.maxabs_scale(df, axis=0, copy=True)
    col = df.columns
    df1 = pd.DataFrame(data=df, columns=col)
    rs = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_seed)
    for train_index, test_index in rs.split(df):
        pass
    traindf = pd.DataFrame(data=df, index=train_index)
    testdf = pd.DataFrame(data=df, index=test_index)
    return traindf, testdf


traindf, testdf = train_test_spliter(df)

X_train = pd.DataFrame(data=traindf, columns=['Canadian Citizen', 'No Response(Citizen)', 'Non-Canadian Citizen',
       'Refugee', 'No Longer In Use', 'Occupied', 'Unoccupied',
       'Employed 0-30 Hours', 'Employed 30+ Hours', 'Gr.10 - Gr.12',
       'Gr.7 - Gr.9', 'Homemaker', 'Kindergarten - Gr.6',
       'No Response(Employment)', 'Permanently Unable to Work',
       'Post Secondary Student', 'Preschool', 'Retired', 'Unemployed',
       'Common Law', 'Married', 'Never Married', 'No Response(Marital)',
       'Separated/Divorced', 'Widowed', 'Bicycle',
       'Car/Truck/Van (as Driver)', 'Car/Truck/Van (as Passenger)',
       'No Response(Transportation)', 'Other', 'Public Transit', 'Walk',
       'Catholic', 'No Response(School)', 'Public'])
X_train = preprocessing.maxabs_scale(X_train, axis=0, copy=True)

y_train = pd.DataFrame(data=traindf, columns=['Assault', 'Break and Enter', 'Homicide',
       'Robbery', 'Sexual Assaults', 'Theft From Vehicle', 'Theft Of Vehicle',
       'Theft Over $5000'])
X_test = pd.DataFrame(data=testdf, columns=['Canadian Citizen', 'No Response(Citizen)', 'Non-Canadian Citizen',
       'Refugee', 'No Longer In Use', 'Occupied', 'Unoccupied',
       'Employed 0-30 Hours', 'Employed 30+ Hours', 'Gr.10 - Gr.12',
       'Gr.7 - Gr.9', 'Homemaker', 'Kindergarten - Gr.6',
       'No Response(Employment)', 'Permanently Unable to Work',
       'Post Secondary Student', 'Preschool', 'Retired', 'Unemployed',
       'Common Law', 'Married', 'Never Married', 'No Response(Marital)',
       'Separated/Divorced', 'Widowed', 'Bicycle',
       'Car/Truck/Van (as Driver)', 'Car/Truck/Van (as Passenger)',
       'No Response(Transportation)', 'Other', 'Public Transit', 'Walk',
       'Catholic', 'No Response(School)', 'Public'])
X_test = preprocessing.maxabs_scale(X_test, axis=0, copy=True)

y_test = pd.DataFrame(data=testdf, columns=['Assault', 'Break and Enter', 'Homicide',
       'Robbery', 'Sexual Assaults', 'Theft From Vehicle', 'Theft Of Vehicle',
       'Theft Over $5000'])

In [4]:
svm.SVR()
# Random forest with multioutput regressor
regr_multisvr = MultiOutputRegressor(svm.SVR())
regr_multisvr.fit(X_train, y_train.values)
svr_result = regr_multisvr.predict(X_test)
svr_result_train = regr_multisvr.predict(X_train)
svr_result

array([[ 0.10034747,  0.10011743,  0.1       , ...,  0.10027212,
         0.10004609,  0.1       ],
       [ 0.51076109,  0.33700983,  0.1       , ...,  1.00801105,
         0.37065477,  0.1       ],
       [ 0.10034747,  0.10011743,  0.1       , ...,  0.10027212,
         0.10004609,  0.1       ],
       ..., 
       [ 0.96248608,  0.63975141,  0.1       , ...,  1.18383043,
         0.57190743,  0.1       ],
       [ 1.88862915,  1.48145783,  0.1       , ...,  2.32506617,
         1.14044364,  0.1       ],
       [ 1.19553472,  0.60215274,  0.1       , ...,  1.10940985,
         0.59446737,  0.1       ]])

In [5]:
crimes = ['Assault', 'Break and Enter', 'Homicide', 'Robbery', 'Sexual Assaults',
       'Theft From Vehicle', 'Theft Of Vehicle', 'Theft Over $5000']

In [6]:
result1 = []
result1_train = []
for i in range(len(crimes)):
    result1.append([np.sqrt(mean_squared_error(y_test[crimes[i]], svr_result[:,i])), crimes[i], 'Test: Single target SVR'])
    result1_train.append([np.sqrt(mean_squared_error(y_train[crimes[i]], svr_result_train[:,i])), crimes[i], 'Train: Single target SVR'])
result1

[[1.9704518353266569, 'Assault', 'Test: Single target SVR'],
 [1.3963277373499654, 'Break and Enter', 'Test: Single target SVR'],
 [0.11609591237233299, 'Homicide', 'Test: Single target SVR'],
 [0.58602606365746224, 'Robbery', 'Test: Single target SVR'],
 [0.4624112189746426, 'Sexual Assaults', 'Test: Single target SVR'],
 [2.05279819912238, 'Theft From Vehicle', 'Test: Single target SVR'],
 [1.1462901944315536, 'Theft Of Vehicle', 'Test: Single target SVR'],
 [0.28181605677304766, 'Theft Over $5000', 'Test: Single target SVR']]

In [7]:
final = [x[0] for x in result1]
final

[1.9704518353266569,
 1.3963277373499654,
 0.11609591237233299,
 0.58602606365746224,
 0.4624112189746426,
 2.05279819912238,
 1.1462901944315536,
 0.28181605677304766]

In [8]:
final_train = [x[0] for x in result1_train]
final_train

[2.0738392488996134,
 1.4611308881498328,
 0.11818371844017735,
 0.61194026421983405,
 0.44363399109128299,
 1.9877347123075662,
 1.1989440573112444,
 0.30255554047556144]